In [4]:
import pandas as pd 

In [5]:
#read data
df19 = pd.read_csv('CPSdata19w-API.csv')
df21 = pd.read_csv('CPSdata21w-API.csv')
df12 = pd.read_csv('CPSdata12w-API.csv')

In [6]:
def create_prev_table(df, disability_flag_col = "PRDISFLG", geo_group_col = "GESTFIPS", count_col_name = "n_dis", total_col_name = "n_pop"):
    # Numerator: # with disability (yes) per geo 
    dis_yes = (
            df[df[disability_flag_col] == 1]
            .groupby(geo_group_col)
            .size()
            .reset_index(name=count_col_name)
        )

        # Denominator: total respondents per geo 
        # (total = 1,2 yes or no)
    total = (
            df[df[disability_flag_col].isin([1, 2])]
            .groupby(geo_group_col)
            .size()
            .reset_index(name=total_col_name)
        )
    
    dis = dis_yes.merge(total, on = 'GESTFIPS')
    dis['Prev'] = dis[count_col_name]/dis[total_col_name]
    dis = dis[['GESTFIPS', 'Prev']]
    return dis

In [7]:
#create prevalence table by state and year
t19 = create_prev_table(df19).rename(columns={'Prev': 'Prev_19'})
t21 = create_prev_table(df21).rename(columns={'Prev': 'Prev_21'})
t12 = create_prev_table(df12).rename(columns={'Prev': 'Prev_12'})

#create paired table with prevalence of disability across states grouped (paired) on years
#Ex. Pair = prevalence in year 2012 vs. year 2019 for California 
m1 = t12.merge(t19, on = "GESTFIPS")
m1 = m1.merge(t21, on = "GESTFIPS")
m1.head()


,GESTFIPS,Prev_12,Prev_19,Prev_21
0,1,0.158212,0.124199,0.128815
1,2,0.118027,0.095745,0.104034
2,4,0.082451,0.116248,0.131040
3,5,0.152000,0.149748,0.172745
4,6,0.104107,0.092992,0.101220


Naive T-test

In [8]:
from scipy.stats import ttest_rel

tstat, pval = ttest_rel(m1['Prev_12'], m1['Prev_21'])


In [9]:
pval

0.0017342812521386644

Testing with Correction

In [10]:
from statsmodels.stats.multitest import multipletests

comparisons = {"2012_vs_2019": ("Prev_12", "Prev_19"), "2019_vs_2021": ("Prev_19", "Prev_21"),"2012_vs_2021": ("Prev_12", "Prev_21")}
results = []

for name, (col1, col2) in comparisons.items():
    
    subset = m1[[col1, col2]]

    t_stat, p_val = ttest_rel(subset[col1], subset[col2])

    results.append({
        "comparison": name,
        "year1_col": col1,
        "year2_col": col2,
        "n_states": len(subset),
        "t_stat": t_stat,
        "p_value": p_val
    })

results_df = pd.DataFrame(results)

# multiple-comparison correction with Holm
reject, p_adj, _, _ = multipletests(results_df["p_value"],alpha=0.05,method="holm")

results_df["p_adj_holm"] = p_adj
results_df["reject_at_0.05"] = reject

print(results_df)

     comparison year1_col year2_col  n_states    t_stat   p_value  p_adj_holm  \
0  2012_vs_2019   Prev_12   Prev_19        51 -1.418598  0.162220    0.162220   
1  2019_vs_2021   Prev_19   Prev_21        51 -2.505161  0.015543    0.031086   
2  2012_vs_2021   Prev_12   Prev_21        51 -3.310342  0.001734    0.005203   

   reject_at_0.05  
0           False  
1            True  
2            True  


In [28]:
adj_r = results_df[['comparison', 't_stat', 'p_value', 'p_adj_holm', 'reject_at_0.05']]

In [30]:
print(adj_r.to_latex())


\begin{tabular}{llrrrr}
\toprule
 & comparison & t_stat & p_value & p_adj_holm & reject_at_0.05 \\
\midrule
0 & 2012_vs_2019 & -1.418598 & 0.162220 & 0.162220 & False \\
1 & 2019_vs_2021 & -2.505161 & 0.015543 & 0.031086 & True \\
2 & 2012_vs_2021 & -3.310342 & 0.001734 & 0.005203 & True \\
\bottomrule
\end{tabular}

